### Check available GPUs

In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Import necessary libraries

In [4]:
import s3fs
import h5py
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.iterative_stratification import IterativeStratification
from sklearn.metrics import precision_score, recall_score, f1_score


### BERT Config

In [5]:
class BertConfig:
    BERT_PREPROCESSOR="https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3"
    BERT_MODEL="https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-12-h-768-a-12/2"
    SEQUENCE_LENGTH=13
    BATCH_SIZE=16
    EPOCHS=3
    LR=2e-5
    NUM_CLASSES=28
    SHUFFLE=42

### Utils function

In [6]:
def load_ds(dataset_type: str, key: str, secret: str, endpoint_url: str):
    s3 = s3fs.S3FileSystem(
        anon=False, 
        key=key, 
        secret=secret, 
        endpoint_url=endpoint_url
    )

    with s3.open(f's3://emotiai/goemotion/{dataset_type}.h5', 'rb') as f:
        h5_file = h5py.File(f, 'r')

        # Stack all tensors into a single tensor (if they have the same shape)
        features = h5_file["features"]
        tensored_features = tf.convert_to_tensor(features)

        labels = h5_file['labels']
        tensored_labels = tf.convert_to_tensor(labels[:], dtype=tf.float32)  
        
    return tf.data.Dataset.from_tensor_slices((tensored_features, tensored_labels)).shuffle(BertConfig.SHUFFLE).batch(BertConfig.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [7]:
def load_ds_10_percent(dataset_type: str, key: str, secret: str, endpoint_url: str, split_size=0.1):
    s3 = s3fs.S3FileSystem(
        anon=False, 
        key=key, 
        secret=secret, 
        endpoint_url=endpoint_url
    )

    with s3.open(f's3://emotiai/goemotion/{dataset_type}.h5', 'rb') as f:
        h5_file = h5py.File(f, 'r')

        features = np.array(h5_file["features"]).reshape(-1, 1)
        labels = np.array(h5_file['labels'][:])
        
        _, _, X_subset, y_subset = iterative_train_test_split(features, labels, test_size=split_size)
        
        tensored_features = tf.convert_to_tensor(X_subset)
        tensored_labels = tf.convert_to_tensor(y_subset, dtype=tf.float32)

    return tf.data.Dataset.from_tensor_slices((tensored_features, tensored_labels)).shuffle(BertConfig.SHUFFLE).batch(BertConfig.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


### Load preprocessed data

In [8]:
ACCESS_KEY="minio_access_key"
SECRET_KEY="minio_secret_key"
ENDPOINT_URL="http://localhost:9000"

In [9]:
train_ds = load_ds("train", key=ACCESS_KEY, secret=SECRET_KEY, endpoint_url=ENDPOINT_URL)
dev_ds = load_ds("dev", key=ACCESS_KEY, secret=SECRET_KEY, endpoint_url=ENDPOINT_URL)
test_ds = load_ds("test", key=ACCESS_KEY, secret=SECRET_KEY, endpoint_url=ENDPOINT_URL)

2025-03-25 10:02:56.290701: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-03-25 10:02:56.290723: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-25 10:02:56.290728: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1742871776.292262 12582399 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1742871776.292691 12582399 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Embed model

In [10]:
def build_bert_preprocessor():
    preprocessor = hub.load(BertConfig.BERT_PREPROCESSOR)
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    tokenize = hub.KerasLayer(preprocessor.tokenize)
    tokenized_input = tokenize(text_input)
    packer = hub.KerasLayer(
        preprocessor.bert_pack_inputs,
        arguments=dict(seq_length=BertConfig.SEQUENCE_LENGTH)
    )
    encoder_inputs = packer([tokenized_input])

    return tf.keras.Model(text_input, encoder_inputs)

### Build model

In [11]:
def build_bert_model(bert_preprocessor, bert_model):
    inputs = tf.keras.layers.Input(shape=(), dtype="string")
    encoder_inputs = bert_preprocessor(inputs)
    bert_outputs = bert_model(encoder_inputs)
    outputs = tf.keras.layers.Dense(BertConfig.NUM_CLASSES, activation="sigmoid")(bert_outputs["pooled_output"])
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

In [12]:
def classification_metrics(average: str = None):
    f1_name = f'_{average}'
    if average == None:
        f1_name = ''

    return [tf.keras.metrics.F1Score(
        name=f'f1_{f1_name}',
        average=average,
    ), tf.keras.metrics.BinaryAccuracy("binary_accuracy"), tf.keras.metrics.Precision(name="precision"), tf.keras.metrics.Recall(name="recall")]

### Transfer learning 
- Feature extraction on 10% of the training dataset
- Fine-tuning with all freeze layer
- Fine-tuning without freezing bottom layer

In [15]:
train_ds_80_percent = load_ds_10_percent("train", key=ACCESS_KEY, secret=SECRET_KEY, endpoint_url=ENDPOINT_URL, split_size=0.8)

In [16]:
# Fine tuning unfreeze on all train dataset
tf.debugging.set_log_device_placement(True)

metrics = classification_metrics("macro")
bert_preprocessor = build_bert_preprocessor()
bert_model = hub.KerasLayer(BertConfig.BERT_MODEL, trainable=True)

model = build_bert_model(bert_preprocessor, bert_model)

model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=BertConfig.LR),
    metrics=metrics
)
model.fit(train_ds_80_percent, epochs=BertConfig.EPOCHS, validation_data=dev_ds)

2025-03-25 10:03:48.990048: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Epoch 1/3
2171/2171 [==============================] - 1833s 837ms/step - loss: 0.1186 - f1__macro: 0.2685 - binary_accuracy: 0.9625 - precision: 0.6286 - recall: 0.2653 - val_loss: 0.1009 - val_f1__macro: 0.3631 - val_binary_accuracy: 0.9667 - val_precision: 0.6883 - val_recall: 0.3804
Epoch 2/3
2171/2171 [==============================] - 1817s 837ms/step - loss: 0.0957 - f1__macro: 0.4016 - binary_accuracy: 0.9682 - precision: 0.7148 - recall: 0.4047 - val_loss: 0.0992 - val_f1__macro: 0.4064 - val_binary_accuracy: 0.9666 - val_precision: 0.6622 - val_recall: 0.4166
Epoch 3/3
2171/2171 [==============================] - 1797s 828ms/step - loss: 0.0833 - f1__macro: 0.4817 - binary_accuracy: 0.9719 - precision: 0.7566 - recall: 0.4877 - val_loss: 0.1047 - val_f1__macro: 0.4107 - val_binary_accuracy: 0.9651 - val_precision: 0.6232 - val_recall: 0.4290


In [ ]:
# # Fine-tuning with all freeze layer

# tf.debugging.set_log_device_placement(True)

# metrics = classification_metrics("macro")
# bert_preprocessor = build_bert_preprocessor()
# bert_model = hub.KerasLayer(BertConfig.BERT_MODEL, trainable=False)

# model = build_bert_model(bert_preprocessor, bert_model)

# model.compile(
#     loss="binary_crossentropy",
#     optimizer=tf.keras.optimizers.Adam(learning_rate=BertConfig.LR),
#     metrics=metrics
# )
# model.fit(train_ds, epochs=BertConfig.EPOCHS, validation_data=dev_ds)

Epoch 1/4


E0000 00:00:1742815497.701410 12079929 meta_optimizer.cc:966] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


2714/2714 [==============================] - 1398s 510ms/step - loss: 0.1889 - f1__macro: 0.0244 - binary_accuracy: 0.9433 - precision: 0.0816 - recall: 0.0340 - val_loss: 0.1469 - val_f1__macro: 0.0343 - val_binary_accuracy: 0.9581 - val_precision: 0.5283 - val_recall: 0.0307
Epoch 2/4
2714/2714 [==============================] - 1455s 536ms/step - loss: 0.1427 - f1__macro: 0.0442 - binary_accuracy: 0.9583 - precision: 0.5641 - recall: 0.0395 - val_loss: 0.1391 - val_f1__macro: 0.0546 - val_binary_accuracy: 0.9584 - val_precision: 0.5568 - val_recall: 0.0484
Epoch 3/4
2714/2714 [==============================] - 1409s 519ms/step - loss: 0.1372 - f1__macro: 0.0619 - binary_accuracy: 0.9587 - precision: 0.5953 - recall: 0.0574 - val_loss: 0.1351 - val_f1__macro: 0.0727 - val_binary_accuracy: 0.9588 - val_precision: 0.5845 - val_recall: 0.0639
Epoch 4/4
2714/2714 [==============================] - 1177s 434ms/step - loss: 0.1340 - f1__macro: 0.0763 - binary_accuracy: 0.9591 - precision: 

### Evaluation 

In [ ]:
def evaluation(model):
    y_pred = model.predict(test_ds)
    
    threshold = 0.5
    y_pred = (y_pred >= threshold).astype(int)
    y_true = np.array([label.numpy() for _, label in test_ds.unbatch()])
    
    precision_per_class = precision_score(y_true, y_pred, average=None)
    recall_per_class = recall_score(y_true, y_pred, average=None)
    f1_per_class = f1_score(y_true, y_pred, average=None)
    classnames = [
        "admiration",
        "amusement",
        "anger",
        "annoyance",
        "approval",
        "caring",
        "confusion",
        "curiosity",
        "desire",
        "disappointment",
        "disapproval",
        "disgust",
        "embarrassment",
        "excitement",
        "fear",
        "gratitude",
        "grief",
        "joy",
        "love",
        "nervousness",
        "optimism",
        "pride",
        "realization",
        "relief",
        "remorse",
        "sadness",
        "surprise",
        "neutral"
    ]


    for i in range(BertConfig.NUM_CLASSES):
        print(f"Class {classnames[i]}:")
        print(f"  Precision: {precision_per_class[i]:.4f}")
        print(f"  Recall:    {recall_per_class[i]:.4f}")
        print(f"  F1-Score:  {f1_per_class[i]:.4f}")
    
    print(f"  Precision macro-average: {precision_per_class[i]:.4f}")
    print(f"  Recall macro-average:    {recall_per_class[i]:.4f}")
    print(f"  F1-Score macro-average:  {f1_per_class[i]:.4f}")

In [18]:
evaluation(model)

340/340 [==============================] - 111s 314ms/step
Class admiration:
  Precision: 0.0935
  Recall:    0.0575
  F1-Score:  0.0713
Class amusement:
  Precision: 0.0667
  Recall:    0.0606
  F1-Score:  0.0635
Class anger:
  Precision: 0.0532
  Recall:    0.0505
  F1-Score:  0.0518
Class annoyance:
  Precision: 0.0598
  Recall:    0.0219
  F1-Score:  0.0320
Class approval:
  Precision: 0.1148
  Recall:    0.0399
  F1-Score:  0.0592
Class caring:
  Precision: 0.0172
  Recall:    0.0074
  F1-Score:  0.0104
Class confusion:
  Precision: 0.0405
  Recall:    0.0196
  F1-Score:  0.0264
Class curiosity:
  Precision: 0.0468
  Recall:    0.0387
  F1-Score:  0.0424
Class desire:
  Precision: 0.0333
  Recall:    0.0120
  F1-Score:  0.0177
Class disappointment:
  Precision: 0.0357
  Recall:    0.0066
  F1-Score:  0.0112
Class disapproval:
  Precision: 0.0159
  Recall:    0.0075
  F1-Score:  0.0102
Class disgust:
  Precision: 0.0292
  Recall:    0.0325
  F1-Score:  0.0308
Class embarrassment:
 

2025-03-25 11:37:53.664962: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/haiduong/miniforge3/envs/emotiai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
